In [163]:
import pandas as pd
import zipfile
from pathlib import Path
import json
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
root_dir = Path(r'C:\Users\97254\PycharmProjects\nlp_project_results\dev_dataset\train_on_80_percent_of_group_names')

In [49]:
EVAL_RESULTS_ZIP = root_dir /'cyber_attrs_eval_by_model_df.zip'
EVAL_GT_ZIP = root_dir /'cyber_attrs_eval_df.zip'

METRICS_JSON = root_dir /'metrics.json'

In [50]:
def extract_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(zip_path.parent)
    return zip_path.with_suffix('.csv')

In [113]:
EVAL_RESULTS_CSV = extract_zip(EVAL_RESULTS_ZIP)
EVAL_GT_CSV = extract_zip(EVAL_GT_ZIP)

In [114]:
EVAL_RESULTS_CSV

WindowsPath('C:/Users/97254/PycharmProjects/nlp_project_results/dev_dataset/train_on_80_percent_of_group_names/cyber_attrs_eval_by_model_df.csv')

In [115]:
EVAL_GT_CSV

WindowsPath('C:/Users/97254/PycharmProjects/nlp_project_results/dev_dataset/train_on_80_percent_of_group_names/cyber_attrs_eval_df.csv')

In [239]:
original_tstr_df = pd.read_csv(EVAL_RESULTS_CSV)[['Unnamed: 0', 'git_repo', 'year', 'filename', 'group_name']]

print(tstr_df.shape)
tstr_df.head()

(3386, 5)


,Unnamed: 0,git_repo,year,filename,group_name
0,0,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
1,1,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
2,2,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
3,3,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]
4,4,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[]


In [240]:
original_tstgt_df = pd.read_csv(EVAL_GT_CSV)[['Unnamed: 0', 'git_repo', 'year', 'filename', 'group_name']]
print(tstgt_df.shape)
tstgt_df.head()

(3386, 5)


,Unnamed: 0,git_repo,year,filename,group_name
0,0,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
1,1,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
2,2,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
3,3,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]
4,4,APT_CyberCriminal_Campagin_Collections,2018,hidden-cobra-targets-turkish-financial-sector-...,[Hidden Cobra]


In [241]:
tstgt_df["group_name"] = original_tstgt_df["group_name"].apply(eval)
tstr_df["group_name"] = original_tstr_df["group_name"].apply(eval)

In [191]:
series_of_list = list(tstgt_df["group_name"])
one_list = [x for _list in series_of_list for x in _list]
all_group_names = set(one_list)
print("num of group_names: ", len(all_group_names))

num of group_names:  116


In [349]:
series_of_list_r = list(tstr_df["group_name"])
one_list_r = [x for _list in series_of_list_r for x in _list]
all_group_names_r = set(one_list_r)
new_names = all_group_names_r-all_group_names
print("num of new_names: ", len(new_names))

num of new_names:  22


In [23]:
labels = ["group_name", "software_name", "technique_name", "tactic"]
print(f"Total lines {len(tstr_df)}")
for label in labels:
    print(f"lines with {label}: {len(tstr_df[tstr_df[label] != str([])])}")

Total lines 3386
lines with group_name: 2832
lines with software_name: 103
lines with technique_name: 34
lines with tactic: 46


In [358]:
def check_if_label_exists(label, line):
    if label in line:
        return True
    return False

In [374]:

verbose_gt_df = tstgt_df.copy()
verbose_r_df = tstr_df.copy()

for group in all_group_names:
    verbose_gt_df[group] = original_tstgt_df.group_name.str.contains(group)
    verbose_r_df[group] = original_tstr_df.group_name.str.contains(group)
    
value_counter = dict()
verbose_lines = dict()
for label_value in all_group_names:
    tp = (verbose_r_df[label_value]==True) & (verbose_gt_df[label_value]==True)
    tn = (verbose_r_df[label_value]==False) &(verbose_gt_df[label_value]==False)
    fp = (verbose_r_df[label_value]==True) & (verbose_gt_df[label_value]==False)
    fn = (verbose_r_df[label_value]==False) & (verbose_gt_df[label_value]==True)
    tp=tp[tp==True]
    tn=tn[tn==True]
    fp=fp[fp==True]
    fn=fn[fn==True]
    counts_ = [len(tp),len(tn), len(fp), len(fn)]
    value_counter[label_value] = counts_
    verbose_lines[label_value] = {'tp': set(tp.index), 'tn': set(tn.index),'fp': set(fp.index),'fn': set(fn.index)}


In [360]:

value_counter_df = pd.DataFrame.from_dict(value_counter).transpose()
value_counter_df.columns =['tp','tn', 'fp','fn']
value_counter_df

,tp,tn,fp,fn
OCEANLOTUS GROUP,0,3385,0,1
dynamite panda,0,3385,0,1
ke3chang,0,3385,0,1
APT30,208,3178,0,0
Suckfly,22,3364,0,0
...,...,...,...,...
turla,0,3378,0,8
Lazarus Group,379,3007,0,0
APT18,23,3363,0,0
LAZARUS GROUP,0,3382,0,4


In [381]:
verbose_gt_df_new = tstgt_df.copy()
verbose_r_df_new = tstr_df.copy()

for group in new_names:
    verbose_gt_df_new[group] = original_tstgt_df.group_name.str.contains(group)
    verbose_r_df_new[group] = original_tstr_df.group_name.str.contains(group)
value_counter_new = dict()
verbose_lines_new = dict()
for label_value in new_names:
    tp = (verbose_r_df_new[label_value]==True) & (verbose_gt_df_new[label_value]==True)
    tn = (verbose_r_df_new[label_value]==False) &(verbose_gt_df_new[label_value]==False)
    fp = (verbose_r_df_new[label_value]==True) & (verbose_gt_df_new[label_value]==False)
    fn = (verbose_r_df_new[label_value]==False) & (verbose_gt_df_new[label_value]==True)
    tp=tp[tp==True]
    tn=tn[tn==True]
    fp=fp[fp==True]
    fn=fn[fn==True]
    counts_ = [len(tp),len(tn), len(fp), len(fn)]
    value_counter_new[label_value] = counts_
    verbose_lines_new[label_value] = {'tp': set(tp.index), 'tn': set(tn.index),'fp': set(fp.index),'fn': set(fn.index)}



In [372]:
len(value_counter_df_new)

22

In [373]:

value_counter_df_new = pd.DataFrame.from_dict(value_counter_new).transpose()
value_counter_df_new.columns =['tp','tn', 'fp','fn']
value_counter_df_new

,tp,tn,fp,fn
ESPIONAGE TARGETED,0,3385,1,0
House,0,3385,1,0
Birds,0,3385,1,0
Contrary,0,3385,1,0
Romance,0,3385,1,0
,0,3386,0,0
NAME,0,3385,1,0
Shell,0,3385,1,0
............................................................................................................................................... 72,0,3385,1,0
APT38,0,3385,1,0


In [384]:
verbose_lines_new['BACKSPACE']

{'tp': set(),
 'tn': {0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155

In [387]:
for word, dict_ in verbose_lines_new.items():
    for key, val in dict_.items():
        if key == 'fp':
            print(word, val)

ESPIONAGE TARGETED {951}
House {1841}
Birds {8}
Contrary {378}
Romance {284}
 set()
NAME {1471}
Shell {43}
............................................................................................................................................... 72 {3058}
APT38 {3352}
DarkHotel group {774}
TG-3279 {1274}
ZJ {2096}
BACKSPACE {2051, 2052, 2467, 2060, 2064, 2096, 2449}
COMPONENT {1438}
RemoveResources {1132}
Propaganda {586}
RIPTIDE {2915, 1205, 1199}
the Storm {1074}
collective set()
Overlapping {742}
The Bear {1071}
